# tweet data from txt to csv

performance on my local machine
- 800Mb takes 11 minutes
- 200Mb takes 3 minutes


In [1]:
import re
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

ss = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
    .getOrCreate()
sc = ss.sparkContext
s  = SQLContext(sc)

t1 = datetime.now()
print(t1)

2020-12-20 12:09:37.334349


In [2]:
#tweets = sc.textFile("./data/tweetstxt/test.txt")
tweets = sc.textFile("./data/tweetstxt/train.txt")
#tweets = sc.textFile("./data/tweetstxt/dev.txt")

In [3]:
tweets_clean = tweets.flatMap(lambda x: x.split(' ')).filter(lambda x:x!='O').filter(lambda x:x!='').filter(lambda x:x!='<STOP>')

In [4]:
tweet_list = tweets_clean.collect()

## list of words to list of tweets 

In [5]:
tweets = []
global counter
counter = -1

def wordsToTweets(x):
    global counter
    if(x=='<START>'):
        counter+=1
        tweets.append('')
    else:
        tweets[counter]+=(x+" ")


In [6]:
for word in tweet_list:
    wordsToTweets(word)

## list of tweets to data table (text,emoji)

In [7]:
tweet_rdd = sc.parallelize(tweets)
tweets = tweet_rdd.zipWithIndex().map(lambda x: (x[1],x[0]))

In [8]:
text_rdd = tweets.map(lambda x: (x[0],re.sub(":.*?:","",x[1])))
emoji_rdd = tweets.map(lambda x: (x[0],re.findall(":.*?:",x[1])))
emoji_rdd1 = emoji_rdd.map(lambda x:(x[0],x[1][0]))
max_emoji = emoji_rdd.map(lambda x: len(x[1])).max()

In [9]:
for i in range(1,max_emoji):
    emoji_rdd2 = emoji_rdd.filter(lambda x:len(x[1])>i).map(lambda x: (x[0],x[1][i]))
    if i==1:
        emoji_rdd3 = emoji_rdd1.union(emoji_rdd2)
    else:
        emoji_rdd3 = emoji_rdd3.union(emoji_rdd2)

rdd_for_df = text_rdd.leftOuterJoin(emoji_rdd3).map(lambda x:(x[1][0][:-1],x[1][1][1:-1]))

## Write to file

takes ± 160 seconds for 200Mb

In [10]:

df = s.createDataFrame(rdd_for_df, ['text','emoji']).distinct()

In [11]:
df.repartition(1).write.csv('./data/test', sep='\t', header=True)

In [12]:
duration = datetime.now()-t1
print(duration)

0:17:20.524952
